In [80]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import scparadise
import pandas as pd
import numpy as np
import pandas as pd
import os
import scvi
import torch

In [81]:
os.chdir('/mnt/c/Users/vadim/scRNA/scParadise/scripts_article')

In [82]:
# Load prepared for training anndata object
adata = sc.read_h5ad('915069db-1df2-49a1-9a9c-2fbd0aa13c81.h5ad')

In [83]:
# Get raw counts from adata.raw
adata = adata.raw.to_adata()

In [84]:
# Convert var_names from ENSG codes to gene names
adata.var.set_index('feature_name', inplace=True)
adata.var_names_make_unique()

In [85]:
# Create adata_train1, adata_train2 and adata_test datasets
adata_test_1 = adata[adata.obs['donor_id'].isin(['HN490'])].copy()
adata_test_2 = adata[adata.obs['donor_id'].isin(['HN492'])].copy()
adata_test_3 = adata[adata.obs['donor_id'].isin(['HN482'])].copy()
adata_test_4 = adata[adata.obs['donor_id'].isin(['HN488'])].copy()
adata_test_5 = adata[adata.obs['donor_id'].isin(['HN485'])].copy()
adata_test_6 = adata[adata.obs['donor_id'].isin(['HN487'])].copy()
adata_test_7 = adata[adata.obs['donor_id'].isin(['HN483'])].copy()
adata_test_8 = adata[adata.obs['donor_id'].isin(['HN489'])].copy()
adata_train_1 = adata[adata.obs['donor_id'].isin(['HN481'])].copy()
adata_train_2 = adata[adata.obs['donor_id'].isin(['HN494'])].copy()
#del adata

In [86]:
adata_train_1.obs['cell_type'].value_counts()

cell_type
epithelial cell                         6747
plasma cell                              868
CD8-positive, alpha-beta T cell          613
fibroblast                               512
myeloid cell                             415
regulatory T cell                        370
endothelial cell                         283
CD4-positive, alpha-beta T cell          282
mural cell                               267
B cell                                   213
mast cell                                 92
natural killer cell                       88
mature alpha-beta T cell                  44
endothelial cell of lymphatic vessel      29
plasmacytoid dendritic cell               19
Name: count, dtype: int64

In [87]:
# Normalize data, find highly variable features
for i in [adata_train_1, adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8]:
    i.layers['counts'] = i.X.copy()
    sc.pp.normalize_total(i, target_sum=None)
    sc.pp.log1p(i)
    i.raw = i

In [88]:
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('Oropharyngeal_cancer/genes_for_AI.csv')
adata_train_1 = adata_train_1[:, genes.genes].copy()

In [89]:
scvi.model.SCVI.setup_anndata(adata_train_1, batch_key="sample_id", layer="counts")

In [90]:
scvi_ref = scvi.model.SCVI(
    adata_train_1
)
scvi_ref.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████████| 400/400 [04:48<00:00,  1.58it/s, v_num=1, train_loss_step=286, train_loss_epoch=255]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████████| 400/400 [04:48<00:00,  1.39it/s, v_num=1, train_loss_step=286, train_loss_epoch=255]


In [91]:
scvi_ref_path = 'Oropharyngeal_cancer/scanvi/model'
scvi_ref.save(scvi_ref_path, overwrite=True)

In [92]:
scanvi_ref = scvi.model.SCANVI.from_scvi_model(
    scvi_ref,
    unlabeled_category="Unknown",
    labels_key='cell_type',
)
# Train scanvi models for selected annotation level
scanvi_ref.train()
scanvi_ref_path = os.path.join('Oropharyngeal_cancer/scanvi/model', 'cell_type')
scanvi_ref.save(scanvi_ref_path, overwrite=True)

INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:15<00:00,  1.49s/it, v_num=1, train_loss_step=254, train_loss_epoch=244]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:15<00:00,  1.51s/it, v_num=1, train_loss_step=254, train_loss_epoch=244]


In [93]:
lst_test_folders = ['HN490', 'HN492', 'HN482', 'HN488', 'HN485', 'HN487', 'HN483', 'HN489']
lst_test_adatas = [adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8]

In [94]:
for adata_test, j in zip(lst_test_adatas, lst_test_folders):
    os.makedirs(os.path.join('Oropharyngeal_cancer/scanvi/reports', j))
    # Train scanvi model for test dataset (adata_test)
    scvi.model.SCANVI.prepare_query_anndata(adata_test, scanvi_ref)
    scanvi_test = scvi.model.SCANVI.load_query_data(adata_test, scanvi_ref)
    scanvi_test.train(
        max_epochs=10
    )
    # Create name for predicted annotation level
    SCANVI_annotation = 'scanvi_cell_type'
    # Predict annotation using pretrained scanvi_test model
    adata_test.obs[SCANVI_annotation] = scanvi_test.predict()
    # Create and save classification report of selected annotation level
    file_save = 'report_test_scanvi_default_' + 'cell_type' + '.csv'
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'cell_type', 
                                          pred_celltype = SCANVI_annotation, 
                                          report_name = file_save,
                                          save_path = os.path.join('Oropharyngeal_cancer/scanvi/reports', j),
                                          save_report=True)

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:13<00:00,  1.33s/it, v_num=1, train_loss_step=346, train_loss_epoch=345]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:13<00:00,  1.33s/it, v_num=1, train_loss_step=346, train_loss_epoch=345]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


...

Epoch 10/10: 100%|██████████████████| 10/10 [00:14<00:00,  1.44s/it, v_num=1, train_loss_step=380, train_loss_epoch=341]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:15<00:00,  1.58s/it, v_num=1, train_loss_step=296, train_loss_epoch=321]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:15<00:00,  1.52s/it, v_num=1, train_loss_step=296, train_loss_epoch=321]
Successfully saved report



In [51]:
# Load prepared for training anndata object
adata = sc.read_h5ad('915069db-1df2-49a1-9a9c-2fbd0aa13c81.h5ad')
# Get raw counts from adata.raw
adata = adata.raw.to_adata()
# Convert var_names from ENSG codes to gene names
adata.var.set_index('feature_name', inplace=True)
adata.var_names_make_unique()
adata.layers['counts'] = adata.X.copy()

In [55]:
del adata.var['name'], adata.var['feature_reference'], adata.var['feature_biotype'], adata.var['feature_length'], adata.var['feature_type']
adata.var['gene_column'] = adata.var_names
adata.var['gene_names'] = adata.var_names

In [52]:
# Save Raw counts for Azimuth and Seurat
import scipy
# Save raw filtered counts and meta data for Azimuth
destination = 'Oropharyngeal_cancer'
adata.obs.to_csv(os.path.join(destination, "metadata.csv"))
# RNA
destination = 'Oropharyngeal_cancer/RNA'
pd.DataFrame(index = adata.var.index).to_csv(os.path.join(destination, "genes.tsv" ), sep = "\t", header=False)
pd.DataFrame(index = adata.obs.index).to_csv(os.path.join(destination, "barcodes.tsv"), sep = "\t", header=False)
scipy.io.mmwrite(os.path.join(destination, "matrix.mtx"), adata.layers['counts'].T)

In [56]:
pd.DataFrame(index = adata.var.index).to_csv(os.path.join(destination, "genes.tsv" ), sep = "\t", header=False)


In [57]:
adata.var

,gene_column,gene_names
feature_name,,
MIR1302-2HG,MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A,FAM138A
OR4F5,OR4F5,OR4F5
ENSG00000238009.6,ENSG00000238009.6,ENSG00000238009.6
ENSG00000239945.1,ENSG00000239945.1,ENSG00000239945.1
...,...,...
ENSG00000277836.1,ENSG00000277836.1,ENSG00000277836.1
ENSG00000278633.1,ENSG00000278633.1,ENSG00000278633.1
ENSG00000276017.1,ENSG00000276017.1,ENSG00000276017.1
